<a href="https://colab.research.google.com/github/OumaimaMarbouh77/DataCampChallenge-AI-for-Meter-Detection/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Dec 16 17:19:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive


Mounted at /content/gdrive


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14639 (delta 4), reused 8 (delta 3), pack-reused 14621
Receiving objects: 100% (14639/14639), 13.25 MiB | 22.51 MiB/s, done.
Resolving deltas: 100% (9959/9959), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-12-16 17:20:52--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   549KB/s    in 6m 15s  

2020-12-16 17:27:08 (423 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/0cfa42.txt     
  inflating: data/obj/0d55b5.jpg     
  inflating: data/obj/0d55b5.txt     
  inflating: data/obj/0d1801.jpg     
  inflating: data/obj/0d1801.txt     
  inflating: data/obj/0da303.jpg     
  inflating: data/obj/0da303.txt     
  inflating: data/obj/0e0106.jpg     
  inflating: data/obj/0e0106.txt     
  inflating: data/obj/0e6201.jpg     
  inflating: data/obj/0e6201.txt     
  inflating: data/obj/0ea4f7.jpg     
  inflating: data/obj/0ea4f7.txt     
  inflating: data/obj/0ec255.jpg     
  inflating: data/obj/0ec255.txt     
  inflating: data/obj/0f1f4d.jpg     
  inflating: data/obj/0f1f4d.txt     
  inflating: data/obj/0f38b6.jpg     
  inflating: data/obj/0f38b6.txt     
  inflating: data/obj/0fe040.jpg     
  inflating: data/obj/0fe040.txt     
  inflating: data/obj/01a650.jpg     
  inflating: data/obj/01a650.txt     
  inflating: data/obj/1a3c33.jpg     
  inflating: data/obj/1a3c33.txt     
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.672028 0.662582 0.246630 0.140523

['15', '0.672028', '0.662582', '0.246630', '0.140523']
0 data/obj/5b4295.txt
0 0.672028 0.662582 0.246630 0.140523
15 0.339583 0.549063 0.379167 0.113125

['15', '0.339583', '0.549063', '0.379167', '0.113125']
1 data/obj/1df3e4.txt
0 0.339583 0.549063 0.379167 0.113125
15 0.489341 0.750431 0.286822 0.257106

['15', '0.489341', '0.750431', '0.286822', '0.257106']
2 data/obj/3cbf1c.txt
0 0.489341 0.750431 0.286822 0.257106
15 0.572077 0.411626 0.131048 0.235887

['15', '0.572077', '0.411626', '0.131048', '0.235887']
3 data/obj/0b69ed.txt
0 0.572077 0.411626 0.131048 0.235887
15 0.488358 0.415033 0.090686 0.190632

['15', '0.488358', '0.415033', '0.090686', '0.190632']
4 data/obj/0f38b6.txt
0 0.488358 0.415033 0.090686 0.190632
15 0.351025 0.491319 0.209987 0.094742

['15', '0.351025', '0.491319', '0.209987', '0.094742']
5 data/obj/2aee00.txt
0 0.351025 0.491319 0.209987 0.094742
15 0.493260 0.560185 0.108456 0.071351

['15', '0.493260', '0.560185',

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/0c5ec4.jpg', 'data/obj/03f766.jpg', 'data/obj/3ef13c.jpg', 'data/obj/0cee7a.jpg', 'data/obj/2cf302.jpg', 'data/obj/5ab82d.jpg', 'data/obj/3c95cc.jpg', 'data/obj/3a6fce.jpg', 'data/obj/4c223a.jpg', 'data/obj/3e0ec1.jpg', 'data/obj/3b0b1f.jpg', 'data/obj/2d72ee.jpg', 'data/obj/4b1339.jpg', 'data/obj/3beb85.jpg', 'data/obj/0b69ed.jpg', 'data/obj/2ce0ef.jpg', 'data/obj/1a17b2.jpg', 'data/obj/3e7092.jpg', 'data/obj/4d5e50.jpg', 'data/obj/5ac118.jpg', 'data/obj/2c65a1.jpg', 'data/obj/2c3aa4.jpg', 'data/obj/5ae8ef.jpg', 'data/obj/0d1801.jpg', 'data/obj/1aa5db.jpg', 'data/obj/5a5105.jpg', 'data/obj/1a3c33.jpg', 'data/obj/0f38b6.jpg', 'data/obj/3af34b.jpg', 'data/obj/4f0739.jpg', 'data/obj/4fc120.jpg', 'data/obj/1be84b.jpg', 'data/obj/03e8fc.jpg', 'data/obj/01a650.jpg', 'data/obj/5b7fc1.jpg', 'data/obj/2d9383.jpg', 'data/obj/05e971.jpg', 'data/obj/5b9884.jpg', 'data/obj/4fc143.jpg', 'data/obj/5b5846.jpg', 'data/obj/4e9620.jpg', 'data/obj/0ea4f7.jpg', 'data/obj/2ee109.jpg', 'data/obj/

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.722258), count: 1, class_loss = 0.003731, iou_loss = 0.055277, total_loss = 0.059009 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.830403), count: 3, class_loss = 0.000639, iou_loss = 0.054898, total_loss = 0.055538 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 132844, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.788971), count: 4, class_loss = 0.047540, iou_loss = 0.110357, total_loss = 0.157897 
v3 (mse loss, Normalizer: (iou